In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import wilcoxon, ttest_rel

In [3]:
colunas_av = ['user_id', 'filme_id', 'rating', 'timestamp']
avaliacao = pd.read_csv('./movielens/u.data', sep='\t', names=colunas_av)
avaliacao.head(20)

,user_id,filme_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [4]:
user = pd.read_csv('./movielens/u.user', sep='|', header = None)
colunas_user = ['user_id', 'idade', 'sexo', 'profissao', 'cep']
user.columns = colunas_user
user.drop(columns = ['cep'], inplace = True)
user.head()

,user_id,idade,sexo,profissao
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [5]:
valores_unicos = user['profissao'].unique()
genero = pd.read_csv('./movielens/u.genre', sep='|', header = None)
colunas_genero = ['genero', 'id_genero']
genero.columns = colunas_genero
genero.head(19)

,genero,id_genero
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4
5,Comedy,5
6,Crime,6
7,Documentary,7
8,Drama,8
9,Fantasy,9


In [6]:
filmes = pd.read_csv('./movielens/u.item', sep='|', encoding='ISO-8859-1', header = None)
colunas_filme = ['filme_id', 'nome_filme','lancamento' ,'3', 'url_filme', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
filmes.columns = colunas_filme
filmes.drop (columns = ['3', 'url_filme'], inplace = True)
filmes.head(20)

,filme_id,nome_filme,lancamento,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
avaliacao.drop(columns= 'timestamp', inplace=True)
avaliacao.head()
df = avaliacao.pivot(index='user_id', columns = 'filme_id', values='rating')

df = df.fillna(0)
df.head()

filme_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_reset = df.reset_index()

# Reformatando o DataFrame para ter 'user_id', 'movie_id', 'rating'
df_formatado = pd.melt(df_reset, id_vars=['user_id'], var_name='movie_id', value_name='rating')
df_formatado = df_formatado[df_formatado['rating'] != 0]
df_formatado.head()

,user_id,movie_id,rating
0,1,1,5.0
1,2,1,4.0
4,5,1,4.0
5,6,1,4.0
9,10,1,4.0


In [41]:
#bins = [0, 1, 2, 3, 4, 5]
#labels = [0, 1, 2, 3, 4]
#df_formatado['rating_class'] = pd.cut(df_formatado['rating'], bins=bins, labels=labels, include_lowest=True)
#df_formatado.head(10)

In [10]:
X = df_formatado[['user_id', 'movie_id']]
y = df_formatado['rating']

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# *KNN*

In [14]:
knn_params = {'n_neighbors': [3, 5, 7],
              'weights': ['uniform', 'distance']}
knn_model = KNeighborsRegressor()

In [15]:
grid_search_knn = GridSearchCV(knn_model, knn_params, scoring='neg_mean_squared_error', cv=5)
grid_search_knn.fit(X_train, y_train)

best_params_knn = grid_search_knn.best_params_

knn_predictions = grid_search_knn.best_estimator_.predict(X_test)

knn_mse = mean_squared_error(y_test, knn_predictions)

print(f"Melhores hiperparâmetros para KNN: {best_params_knn}")
print(f"KNN MSE: {knn_mse}")

Melhores hiperparâmetros para KNN: {'n_neighbors': 7, 'weights': 'uniform'}
KNN MSE: 1.2203510204081636


In [16]:
threshold = 2
y_test_binary = (y_test >= threshold).astype(int)
knn_predictions_binary = (knn_predictions >= threshold).astype(int)

In [17]:
accuracy_knn = accuracy_score(y_test_binary, knn_predictions_binary)
precision_knn = precision_score(y_test_binary, knn_predictions_binary, average='weighted')
recall_knn = recall_score(y_test_binary, knn_predictions_binary, average='weighted')
f1_knn = f1_score(y_test_binary, knn_predictions_binary, average='weighted')

# Exiba as métricas
print(f"KNN Accuracy: {accuracy_knn}")
print(f"KNN Precision: {precision_knn}")
print(f"KNN Recall: {recall_knn}")
print(f"KNN F1 Score: {f1_knn}")

KNN Accuracy: 0.93785
KNN Precision: 0.9131879575003681
KNN Recall: 0.93785
KNN F1 Score: 0.9153043769200904


# *SVM*

In [11]:
svm_params = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'rbf', 'poly']}
svm_model = SVR()

In [ ]:
#grid_search_svm = GridSearchCV(svm_model, svm_params, scoring='neg_mean_squared_error', cv=5)
#grid_search_svm.fit(X_train, y_train)

#best_params_svm = grid_search_svm.best_params_

#svm_predictions = grid_search_svm.best_estimator_.predict(X_test)

#svm_mse = mean_squared_error(y_test, svm_predictions)

#print(f"Melhores hiperparâmetros para SVM: {best_params_svm}")
#print(f"SVM MSE: {svm_mse}")

In [12]:
svm_model.fit(X_train, y_train)

# Faça predições no conjunto de teste
svm_predictions = svm_model.predict(X_test)

# Calcule o MSE
svm_mse = mean_squared_error(y_test, svm_predictions)
print(f"SVM MSE: {svm_mse}")

SVM MSE: 1.2689378814531813


In [18]:
svm_predictions_binary = (svm_predictions >= threshold).astype(int)

In [20]:
accuracy_svm = accuracy_score(y_test_binary, svm_predictions_binary)
precision_svm = precision_score(y_test_binary, svm_predictions_binary, average='weighted')
recall_svm = recall_score(y_test_binary, svm_predictions_binary, average='weighted')
f1_svm = f1_score(y_test_binary, svm_predictions_binary, average='weighted')

# Exiba as métricas
print(f"SVM Accuracy: {accuracy_svm}")
print(f"SVM Precision: {precision_svm}")
print(f"SVM Recall: {recall_svm}")
print(f"SVM F1 Score: {f1_svm}")

SVM Accuracy: 0.93825
SVM Precision: 0.8803130625
SVM Recall: 0.93825
SVM F1 Score: 0.9083586353669547


c:\Users\naanv\OneDrive\Documentos\Programa\IC\Recomendacao\tomy\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# *NB*

In [21]:
nb_model = GaussianNB()

In [22]:
nb_model.fit(X_train, y_train)

nb_predictions = nb_model.predict(X_test)

In [23]:
nb_predictions_binary = (nb_predictions >= threshold).astype(int)

In [24]:
accuracy_nb = accuracy_score(y_test_binary, nb_predictions_binary)
precision_nb = precision_score(y_test_binary, nb_predictions_binary, average='weighted')
recall_nb = recall_score(y_test_binary, nb_predictions_binary, average='weighted')
f1_nb = f1_score(y_test_binary, nb_predictions_binary, average='weighted')

# Exiba as métricas
print(f"Naive Bayes Accuracy: {accuracy_nb}")
print(f"Naive Bayes Precision: {precision_nb}")
print(f"Naive Bayes Recall: {recall_nb}")
print(f"Naive Bayes F1 Score: {f1_nb}")

Naive Bayes Accuracy: 0.9339
Naive Bayes Precision: 0.8971906399313601
Naive Bayes Recall: 0.9339
Naive Bayes F1 Score: 0.9098811535362958


# *Wilcoxon*

In [34]:
def wilcoxon_test_all_metrics(metrics_algo1, metrics_algo2):
    statistics = []
    p_values = []

    for metric_algo1, metric_algo2 in zip(metrics_algo1, metrics_algo2):

        differences = metric_algo1 - metric_algo2

        statistic, p_value = wilcoxon([differences])

        statistics.append(statistic)
        p_values.append(p_value)

    return statistics, p_values

In [37]:
metrics_knn = [accuracy_knn, precision_knn, recall_knn, f1_knn]
metrics_svm = [accuracy_svm, precision_svm, recall_svm, f1_svm]
metrics_nb = [accuracy_nb, precision_nb, recall_nb, f1_nb]

statistics_knn_svm, p_values_knn_svm = wilcoxon_test_all_metrics(metrics_knn, metrics_svm)
statistics_knn_nb, p_values_knn_nb = wilcoxon_test_all_metrics(metrics_knn, metrics_nb)
statistics_svm_nb, p_values_svm_nb = wilcoxon_test_all_metrics(metrics_svm, metrics_nb)

In [39]:
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-score']
for metric_name, statistic, p_value in zip(metrics_names, statistics_knn_svm, p_values_knn_svm):
    print(f"Wilcoxon Statistic e p-value para KNN vs. SVM ({metric_name}): {statistic}, {p_value}")
print()

for metric_name, statistic, p_value in zip(metrics_names, statistics_knn_nb, p_values_knn_nb):
    print(f"Wilcoxon Statistic e p-value para KNN vs. NB ({metric_name}): {statistic}, {p_value}")
print()

for metric_name, statistic, p_value in zip(metrics_names, statistics_svm_nb, p_values_svm_nb):
    print(f"Wilcoxon Statistic e p-value para SVM vs. NB ({metric_name}): {statistic}, {p_value}")
print()

Wilcoxon Statistic e p-value para KNN vs. SVM (Accuracy): 0.0, 1.0
Wilcoxon Statistic e p-value para KNN vs. SVM (Precision): 0.0, 1.0
Wilcoxon Statistic e p-value para KNN vs. SVM (Recall): 0.0, 1.0
Wilcoxon Statistic e p-value para KNN vs. SVM (F1-score): 0.0, 1.0

Wilcoxon Statistic e p-value para KNN vs. NB (Accuracy): 0.0, 1.0
Wilcoxon Statistic e p-value para KNN vs. NB (Precision): 0.0, 1.0
Wilcoxon Statistic e p-value para KNN vs. NB (Recall): 0.0, 1.0
Wilcoxon Statistic e p-value para KNN vs. NB (F1-score): 0.0, 1.0

Wilcoxon Statistic e p-value para SVM vs. NB (Accuracy): 0.0, 1.0
Wilcoxon Statistic e p-value para SVM vs. NB (Precision): 0.0, 1.0
Wilcoxon Statistic e p-value para SVM vs. NB (Recall): 0.0, 1.0
Wilcoxon Statistic e p-value para SVM vs. NB (F1-score): 0.0, 1.0



# *Teste t de Student*

In [95]:
statistic_knn_svm, p_value_knn_svm = ttest_rel(metrics_knn, metrics_svm)
statistic_knn_nb, p_value_knn_nb = ttest_rel(metrics_knn, metrics_nb)
statistic_svm_nb, p_value_svm_nb = ttest_rel(metrics_svm, metrics_nb)

In [96]:
print(f"T-Test Statistic e p-value para KNN vs. SVM: {statistic_knn_svm}, {p_value_knn_svm}")
print(f"T-Test Statistic e p-value para KNN vs. NB: {statistic_knn_nb}, {p_value_knn_nb}")
print(f"T-Test Statistic e p-value para SVM vs. NB: {statistic_svm_nb}, {p_value_svm_nb}")

T-Test Statistic e p-value para KNN vs. SVM: 1.2350368136495622, 0.30472427832847354
T-Test Statistic e p-value para KNN vs. NB: 2.5190733621994355, 0.08624692575884578
T-Test Statistic e p-value para SVM vs. NB: -0.4838026388265858, 0.6616436373642465
